# Análisis de datos

Importa las herramientas necesarias para el análisis. 
* pandas permite trabajar con datos tabulares de manera eficiente
* matplotlib.pyplot se utiliza para generar gráficos,
* re facilita extraer números de cadenas de texto,
* json sirve para guardar los resultados en un formato estructurado que puede ser reutilizado en otros procesos o sistemas.

In [5]:
# Librerías
import pandas as pd
import matplotlib.pyplot as plt
import re
import json

Toma valores de cobertura del territorio que vienen como texto y devuelve solo el porcentaje. Esto permite manejar los datos de vegetación y áreas antropogénicas de manera numérica para poder calcular tendencias y balances.

In [7]:
def analizar_serie_temporal():
    df_coverage = pd.read_csv("coverage_completo_1985_2024.csv")

    tendencias = calcular_tendencias(df_coverage)
    puntos_inflexion = identificar_puntos_inflexion(tendencias)
    
    generar_historia_territorial(tendencias, puntos_inflexion)
    exportar_datos(tendencias, puntos_inflexion)
    generar_reporte(tendencias)
    crear_visualizaciones_separadas(tendencias)

Se combinan las categorías de cobertura vegetal y antropogénica para obtener un resumen anual de la situación del territorio. La columna diferencia refleja el balance ecológico de cada año, indicando si la vegetación supera a las áreas humanas o viceversa.

In [9]:
def calcular_tendencias(df_coverage):
    tendencias = []
    for año in sorted(df_coverage['year'].unique()):
        datos_año = df_coverage[df_coverage['year'] == año].iloc[0]
        vegetacion = extraer_porcentaje(datos_año['Natural herbaceous and shrub vegetation'])
        bosques = extraer_porcentaje(datos_año['Forests'])
        agricola = extraer_porcentaje(datos_año['Agricultural and livestock area'])
        urbano = extraer_porcentaje(datos_año['Non-vegetated area'])
        tendencias.append({
            'year': año,
            'vegetacion_total': vegetacion + bosques,
            'areas_antropogenicas': agricola + urbano,
            'diferencia': (vegetacion + bosques) - (agricola + urbano)
        })
    return pd.DataFrame(tendencias)

Se identifica los años en los que se producen cambios importantes en el balance ecológico, señalando si el cambio corresponde a una mejora o un deterioro significativo del territorio. Esto ayuda a resaltar periodos críticos de transformación.

In [11]:
def identificar_puntos_inflexion(tendencias):
    puntos = []
    for i in range(1, len(tendencias)):
        cambio = tendencias.iloc[i]['diferencia'] - tendencias.iloc[i-1]['diferencia']
        if abs(cambio) > 5:
            puntos.append({
                'year': tendencias.iloc[i]['year'],
                'cambio': cambio,
                'tipo': 'deterioro' if cambio < 0 else 'mejora'
            })
    return puntos

Aquí se resumen los principales hallazgos del análisis, mostrando el periodo cubierto, el cambio neto de vegetación y el estado final del equilibrio ecológico, además de destacar los puntos de inflexión más relevantes para facilitar la interpretación de los datos.

In [13]:
def generar_historia_territorial(tendencias, puntos_inflexion):
    primer_año = tendencias['year'].min()
    ultimo_año = tendencias['year'].max()
    cambio_total = tendencias.iloc[-1]['vegetacion_total'] - tendencias.iloc[0]['vegetacion_total']
    print(f"Periodo analizado: {primer_año}-{ultimo_año}")
    print(f"Cambio total de vegetación: {cambio_total:+.1f}%")
    print(f"Equilibrio ecológico final: {'positivo' if tendencias.iloc[-1]['diferencia'] > 0 else 'negativo'}")
    if puntos_inflexion:
        print("Primeros 3 puntos de inflexión:")
        for punto in puntos_inflexion[:3]:
            print(f"  • {punto['year']}: {'mejora' if punto['tipo']=='mejora' else 'deterioro'} de {punto['cambio']:+.1f}%")

Se guardan los datos procesados en formatos JSON y CSV, permitiendo su uso posterior en otros análisis, visualizaciones o sistemas de información sin necesidad de recalcular todo desde cero.

In [15]:
def exportar_datos(tendencias, puntos_inflexion):
    datos = {
        "metadata": {
            "periodo": f"{tendencias['year'].min()}-{tendencias['year'].max()}",
            "total_años": len(tendencias),
            "unidades": "porcentaje"
        },
        "tendencias_anuales": tendencias.to_dict('records'),
        "puntos_clave": puntos_inflexion
    }
    with open('analisis_territorial.json', 'w', encoding='utf-8') as f:
        json.dump(datos, f, indent=2, ensure_ascii=False)
    tendencias.to_csv('datos_analisis_estructurado.csv', index=False)

Se genera un documento de texto que resume los hallazgos principales de manera estructurada, incluyendo los valores máximos y mínimos, así como el estado final del territorio, facilitando su presentación o revisión.

In [17]:
def generar_reporte(tendencias):
    primer_año = tendencias['year'].min()
    ultimo_año = tendencias['year'].max()
    cambio_veg = tendencias.iloc[-1]['vegetacion_total'] - tendencias.iloc[0]['vegetacion_total']
    reporte = f"""
Análisis Territorial {primer_año}-{ultimo_año}

Periodo: {primer_año}-{ultimo_año} ({len(tendencias)} años)
Cambio neto vegetación: {cambio_veg:+.1f}%
Estado final: {'Equilibrio ecológico positivo' if tendencias.iloc[-1]['diferencia'] > 0 else 'Presión antrópica dominante'}

Vegetación máxima: {tendencias['vegetacion_total'].max():.1f}% en {tendencias.loc[tendencias['vegetacion_total'].idxmax(), 'year']}
Vegetación mínima: {tendencias['vegetacion_total'].min():.1f}% en {tendencias.loc[tendencias['vegetacion_total'].idxmin(), 'year']}
Presión antrópica máxima: {tendencias['areas_antropogenicas'].max():.1f}% en {tendencias.loc[tendencias['areas_antropogenicas'].idxmax(), 'year']}
"""
    with open('reporte_territorial.txt', 'w', encoding='utf-8') as f:
        f.write(reporte)

Se produce cuatro gráficos independientes que permiten visualizar la evolución de la vegetación frente a las áreas humanas, el balance ecológico, los cambios acumulados de vegetación y la velocidad de cambio anual, proporcionando una representación clara de la dinámica territorial a lo largo del tiempo.

In [19]:
def crear_visualizaciones_separadas(tendencias):
    # 1. Evolución Vegetación vs. Área Antrópica
    plt.figure(figsize=(10,6))
    plt.plot(tendencias['year'], tendencias['vegetacion_total'], label='Área Vegetal', color='#2ecc71', marker='o')
    plt.plot(tendencias['year'], tendencias['areas_antropogenicas'], label='Área Antrópica', color='#e74c3c', marker='s')
    plt.ylabel('Porcentaje del Territorio (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('grafico_vegetacion_vs_antropica.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Balance ecológico
    plt.figure(figsize=(10,6))
    plt.plot(tendencias['year'], tendencias['diferencia'], color='#27ae60')
    plt.ylabel('Balance (%)')
    plt.grid(True, alpha=0.3)
    plt.savefig('grafico_balance_ecologico.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. Cambio acumulado de vegetación
    cambio_acumulado = tendencias['vegetacion_total'] - tendencias['vegetacion_total'].iloc[0]
    plt.figure(figsize=(10,6))
    plt.bar(tendencias['year'], cambio_acumulado, color=['#e74c3c' if x < 0 else '#2ecc71' for x in cambio_acumulado], alpha=0.7)
    plt.ylabel('Cambio Acumulado (%)')
    plt.grid(True, alpha=0.3)
    plt.savefig('grafico_cambio_acumulado.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. Velocidad de cambio anual
    velocidad = tendencias['vegetacion_total'].diff()
    plt.figure(figsize=(10,6))
    plt.bar(tendencias['year'], velocidad, color=['#c0392b' if x < 0 else '#16a085' for x in velocidad], alpha=0.7)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    plt.ylabel('Cambio Anual (% puntos)')
    plt.grid(True, alpha=0.3)
    plt.savefig('grafico_velocidad_cambio.png', dpi=300, bbox_inches='tight')
    plt.close()

def extraer_porcentaje(texto):
    if pd.isna(texto) or texto == '-  ha ( - %)':
        return 0.0
    match = re.search(r'\(([\d.-]+)%\)', str(texto))
    return float(match.group(1)) if match else 0.0

if __name__ == "__main__":
    analizar_serie_temporal()


Periodo analizado: 1985-2024
Cambio total de vegetación: -4.7%
Equilibrio ecológico final: positivo
